# Building custom Neural Networks in Compyute

In [ ]:
import compyute as cp

## Prepare the Data

In [ ]:
# ! pip install pandas

In [ ]:
import pandas as pd
from compyute.preprocessing import split_train_val_test, normalize

# download the data
url = "https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv"
data = pd.read_csv(url)

# encode the targets
data["variety"] = data["variety"].astype("category").cat.codes
data_tensor = cp.tensor(data.to_numpy())

# split the data into train, val, test
train, val, test = split_train_val_test(data_tensor, ratio_val=0.25, ratio_test=0.25)

# split features from targets
X_train, y_train = train[:, :-1], train[:, -1].to_int()
X_val, y_val = val[:, :-1], val[:, -1].to_int()
X_test, y_test = test[:, :-1], test[:, -1].to_int()

# normalize features
X_train = normalize(X_train, dim=0, low=-1, high=1)
X_val = normalize(X_val, dim=0, low=-1, high=1)
X_test = normalize(X_test, dim=0, low=-1, high=1)

print (f'{X_train.shape=}')
print (f'{y_train.shape=}')
print (f'{X_val.shape=}')
print (f'{y_val.shape=}')
print (f'{X_test.shape=}')
print (f'{y_test.shape=}')

## Build a custom Model Architecture
If you want to define very specifically what the forward and backward pass of your model should look like, you can build your own custom model instead of using the predefined sequential model.

In [ ]:
from compyute import nn

class MyCustomModel(nn.Module):  # inherit from nn.Module
    def __init__(self):
        super().__init__()

        # define your layers
        self.lin1 = nn.Linear(4, 16)
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(16, 3)

    # define the forward pass
    @nn.Module.register_forward
    def forward(self, x):
        x = self.lin1(x)
        x = self.relu(x)
        x = self.lin2(x)
        return x

    # define the backward pass
    @nn.Module.register_backward
    def backward(self, dy):
        dy = self.lin2.backward(dy)
        dy = self.relu.backward(dy)
        dy = self.lin1.backward(dy)
        return dy
        

model = MyCustomModel()

In [ ]:
summary = cp.nn.utils.get_module_summary(model, input_shape=(4,))
print(summary)

## Write a custom Training Loop

You can also write a custom training loop instead of using the `Trainer`.

In [ ]:
epochs = 10000
batch_size = X_train.shape[0]

train_dl = nn.utils.Dataloader((X_train, y_train), batch_size=batch_size)
val_dl = nn.utils.Dataloader((X_val, y_val), batch_size=batch_size)

loss_fn = nn.CrossEntropy()
optim = nn.optimizers.SGD(model.get_parameters(), lr=1e-2)

for e in range(epochs):
    # training
    model.training()
    for x, y in train_dl():
        # forward pass
        y_pred = model(x)
        _ = loss_fn(y_pred, y)

        # backward pass
        model.backward(loss_fn.backward())  # compute new gradients
        optim.step()  # update parameters
        optim.reset_grads()  # reset all gradients
    
    # validiation
    model.inference()
    with nn.no_caching():  # disable caching values for gradient computation
        val_loss = 0
        for x, y in val_dl():
            y_pred = model(x)
            val_loss += loss_fn(y_pred, y).item()
        val_loss /= len(val_dl)
        print(f"epoch {e}: {val_loss=:.4f}")